In [1]:
import chess
import chess.polyglot
import chess.svg
import chess.pgn
import chess.engine
from IPython.display import SVG
from time import sleep
import datetime

from tables import knightstable, pawntable, bishopstable, rookstable, queenstable, kingstable
board = chess.Board()


def evaluate_board():
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    if board.is_stalemate():
        return 0
    if board.is_insufficient_material():
        return 0

    wp = len(board.pieces(chess.PAWN, chess.WHITE)) #white pawn
    bp = len(board.pieces(chess.PAWN, chess.BLACK)) #black pawn
    wn = len(board.pieces(chess.KNIGHT, chess.WHITE)) #white knight
    bn = len(board.pieces(chess.KNIGHT, chess.BLACK)) #black knight
    wb = len(board.pieces(chess.BISHOP, chess.WHITE)) #white bishop
    bb = len(board.pieces(chess.BISHOP, chess.BLACK)) #black bishop
    wr = len(board.pieces(chess.ROOK, chess.WHITE)) #white rook
    br = len(board.pieces(chess.ROOK, chess.BLACK)) #black rook
    wq = len(board.pieces(chess.QUEEN, chess.WHITE)) #white queen
    bq = len(board.pieces(chess.QUEEN, chess.BLACK)) #black  queen
    #king need not be counted, as game will end if king is in checkmate.
    #I.e no situation with less or more than 1 king per side will happen

    #value of: Pawn: 100, Knight: 320, Bishop: 330,       Rook: 500,    Queen: 900
    material = 100 * (wp-bp) + 320 * (wn-bn) + 330 * (wb-bb) + 500 * (wr-br) + 900 * (wq-bq)
    #^Material score is the sum of the difference between amount of white and black pieces multiplied by their respective values

    pawnsq = sum([pawntable[i] for i in board.pieces(chess.PAWN, chess.WHITE)])
    #take the sum of all pawntable values according to positions of the pawns. Use list comprehension, and board.pieces
    #Which lists position of every white pawn here.
    pawnsq = pawnsq + sum([-pawntable[chess.square_mirror(i)] for i in board.pieces(chess.PAWN, chess.BLACK)])
    #Do the same now with black pieces, though we have to mirror and invert them (as explained in tables.py.)

    knightsq = sum([knightstable[i] for i in board.pieces(chess.KNIGHT, chess.WHITE)])
    knightsq = knightsq + sum([-knightstable[chess.square_mirror(i)] for i in board.pieces(chess.KNIGHT, chess.BLACK)])

    bishopsq = sum([bishopstable[i] for i in board.pieces(chess.BISHOP, chess.WHITE)])
    bishopsq = bishopsq + sum([-bishopstable[chess.square_mirror(i)] for i in board.pieces(chess.BISHOP, chess.BLACK)])

    rooksq = sum([rookstable[i] for i in board.pieces(chess.ROOK, chess.WHITE)])
    rooksq = rooksq + sum([-rookstable[chess.square_mirror(i)] for i in board.pieces(chess.ROOK, chess.BLACK)])

    queensq = sum([queenstable[i] for i in board.pieces(chess.QUEEN, chess.WHITE)])
    queensq = queensq + sum([-queenstable[chess.square_mirror(i)] for i in board.pieces(chess.QUEEN, chess.BLACK)])

    kingsq = sum([kingstable[i] for i in board.pieces(chess.KING, chess.WHITE)])
    kingsq = kingsq + sum([-kingstable[chess.square_mirror(i)] for i in board.pieces(chess.KING, chess.BLACK)])

    eval = material + pawnsq + knightsq + bishopsq + rooksq + queensq + kingsq
    if board.turn: #board.turn returns true if it's white's turn
        return eval
    else:
        return -eval
    #If and proportional to if and how much the value is positive, white is in the lead. Black will be the inverse.
    #So returns the inverse if it's black's turn

def selectmove(depth):
    try:
        move = chess.polyglot.MemoryMappedReader("./books/human.bin").weighted_choice(board).move
        return move    
    except:
        bestMove = chess.Move.null()
        bestValue = -99999
        alpha = -100000
        beta = 100000
        for move in board.legal_moves:
            board.push(move)
            boardValue = -alphabeta(-beta, -alpha, depth - 1)
            if boardValue > bestValue:
                bestValue = boardValue
                bestMove = move
            if (boardValue > alpha):
                alpha = boardValue
            board.pop()
        return bestMove

def alphabeta(alpha, beta, depthleft):
    bestscore = -9999
    if (depthleft == 0):
        return quiesce(alpha, beta)
    for move in board.legal_moves:
        board.push(move)
        score = -alphabeta(-beta, -alpha, depthleft - 1)
        board.pop()
        if (score >= beta):
            return score
        if (score > bestscore):
            bestscore = score
        if (score > alpha):
            alpha = score
    return bestscore

def quiesce(alpha, beta):
    stand_pat = evaluate_board()
    if (stand_pat >= beta):
        return beta
    if (alpha < stand_pat):
        alpha = stand_pat

    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)
            score = -quiesce(-beta, -alpha)
            board.pop()

            if (score >= beta):
                return beta
            if (score > alpha):
                alpha = score
    return alpha 

Run below (alternating) for AI vs HUMAN

In [ ]:
board.push(selectmove(3))
SVG(chess.svg.board(board=board,size=400))

In [ ]:
board.push_san("a1b2")
SVG(chess.svg.board(board=board,size=400))

Run below for AI vs AI

In [ ]:
count = 0
while not board.is_game_over(claim_draw=True):
    mov = selectmove(2)
    board.push(mov)
    print(board)
    count += 1
    print(count)

print(board.is_game_over(claim_draw=True))
SVG(chess.svg.board(board=board,size=400))